In [13]:
import os
import sys
import numpy as np
import xarray as xr
import pandas as pd
import hvplot.xarray  # For visualization

import emit_tools as et
from emit_tools import emit_xarray
import earthaccess
import json
from sklearn.cross_decomposition import PLSRegression

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [14]:
earthaccess.login(persist=True)

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
earthaccess generated a token for CMR with expiration on: 06/08/2024
earthaccess generated a token for CMR with expiration on: 06/08/2024
Using .netrc file for EDL


Replace the url below with the link to the EMIT file desired using earthaccess. 

Get an HTTPS Session using your earthdata login, set a local path to save the file, and download the granule asset - This may take a while, the reflectance file is approximately 1.8 GB.

In [15]:
url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230826T175001_2323812_003/EMIT_L2A_RFL_001_20230826T175001_2323812_003.nc'

In [19]:
# Get requests https Session using Earthdata Login Info
fs = earthaccess.get_requests_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = f'data/{granule_asset_id}'
# Download the Granule Asset if it doesn't exist
if not os.path.isfile(fp):
    with fs.get(url,stream=True) as src:
        with open(fp,'wb') as dst:
            for chunk in src.iter_content(chunk_size=64*1024*1024):
                dst.write(chunk)

In [5]:
print(granule_asset_id)

EMIT_L2A_RFL_001_20230826T175001_2323812_003.nc


Load up the EMIT image, normalize it as instructed by Ting

In [20]:
# Load the EMIT dataset with orthorectification
ds_geo = emit_xarray(fp, ortho=True)

# Normalize the reflectance data across wavelengths
norm = np.sqrt((ds_geo.reflectance**2).sum(dim="wavelengths"))
normalized_reflectance = ds_geo.reflectance / norm

Begin loading and formatting JSON

In [21]:
#Load the JSON nitrogen AVIRIS map

# Opening JSON file
f = open(f'../trait_data/PLSR_raw_coef_Nitrogen_1000_2400.json')
 
# returns JSON object as 
# a dictionary
plsr = json.load(f)
 
# Iterating through the json
# list

print(plsr['name'])
print(plsr['spectrometer'])
print(plsr['type'])
print(plsr['units'])
print(plsr['wavelength_units'])
wavelengths = plsr['wavelengths']
min_wavelength = min(wavelengths)
max_wavelength = max(wavelengths)
n_wavelength = len(wavelengths)
print("Wavelength range of", n_wavelength,": ", min_wavelength, "-", max_wavelength)
for i in plsr['wavelengths']:
    print(i)

 
# Closing file
f.close()

Nitrogen
avc+neon
plsr
mg/g
nanometers
Wavelength range of 95 :  1000.0 - 2400.0
1000.0
1010.0
1020.0
1030.0
1040.0
1050.0
1060.0
1070.0
1080.0
1090.0
1100.0
1110.0
1120.0
1130.0
1140.0
1150.0
1160.0
1170.0
1180.0
1190.0
1200.0
1210.0
1220.0
1230.0
1240.0
1250.0
1260.0
1470.0
1480.0
1490.0
1500.0
1510.0
1520.0
1530.0
1540.0
1550.0
1560.0
1570.0
1580.0
1590.0
1600.0
1610.0
1620.0
1630.0
1640.0
1650.0
1660.0
1670.0
1680.0
1690.0
1700.0
1710.0
1720.0
1730.0
1740.0
1750.0
1760.0
1770.0
1780.0
2050.0
2060.0
2070.0
2080.0
2090.0
2100.0
2110.0
2120.0
2130.0
2140.0
2150.0
2160.0
2170.0
2180.0
2190.0
2200.0
2210.0
2220.0
2230.0
2240.0
2250.0
2260.0
2270.0
2280.0
2290.0
2300.0
2310.0
2320.0
2330.0
2340.0
2350.0
2360.0
2370.0
2380.0
2390.0
2400.0


In [22]:
# Get PLSR model stuff
c = plsr["model"]["coefficients"] # Coefficients
n = plsr["model"]["components"] # Number of components
i = plsr["model"]["intercepts"] # Intercept

Build the array

In [23]:
# Simulated flightline shape: bands, rows, columns
array_dimensions = (95, 1000, 2400) # Only load wavelengths listed in json object, ~95 total bands
X = np.random.rand(*array_dimensions)
print(X.shape[0])
print(X.shape[1])
print(X.shape[2])

95
1000
2400


In [24]:
# Reshape the array to the expected model input (wavelength pixel values, 
#number of pixels)/(95, 6000)
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]).T


In [25]:
# PLSR Stuff
pls_model = PLSRegression(n_components=n, copy=False)
pls_model.intercept_ = np.array(i)
pls_model.coef_ = np.array(c)
pls_model._x_mean = np.mean(X)
pls_model._x_std = np.std(X)

In [26]:
# Make Predictions
predictions = pls_model.predict(X) 

type: 'PLSRegression' object has no attribute '_predict_1d'

Now you just have to reshape the predicted values back into their original form and write it into a GeoTiff!

In [78]:
# Reshape X to its original shape
X_restored = X.T.reshape(array_dimensions)

In [ ]:
#Create GeoTiff!